In [1]:

import pandas as pd

# Actually filtering the dataset to passages and passage text

In [2]:
import duckdb
from datasets import load_dataset
import pandas as pd

# Step 1: Load the dataset from Hugging Face
dataset = load_dataset("microsoft/ms_marco", "v1.1")  # Replace with your actual dataset name

# Step 2: Convert to Pandas DataFrame
df = dataset['train'].to_pandas()

# Step 3: Create a DuckDB connection
con = duckdb.connect()

# Optionally: Register the DataFrame as a DuckDB table
con.execute("CREATE TABLE v11_train AS SELECT * FROM df")

# Step 4: Query the dataset
query = """
SELECT *,
       passages->>'passage_text' AS passage_text,
       passages->'is_selected' AS is_selected,
       passages->>'url' AS url
FROM v11_train;
"""

# Execute the query
result = con.execute(query).fetchdf()

# Step 5: Save the results to a CSV file
result.to_csv('your_file.csv', index=False)

print("Data has been saved to 'your_file.csv'.")

# Close the DuckDB connection
con.close()



c:\Users\omare\anaconda3\envs\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data has been saved to 'your_file.csv'.


In [3]:
result

,answers,passages,query,query_id,query_type,wellFormedAnswers,passage_text,is_selected,url
0,[Results-Based Accountability is a disciplined...,"{'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]...",what is rba,19699,description,[],"[""Since 2007, the RBA's outstanding reputation...","[0,0,0,0,0,1,0,0,0,0]","[""https://en.wikipedia.org/wiki/Reserve_Bank_o..."
1,[Yes],"{'is_selected': [0, 1, 0, 0, 0, 0, 0], 'passag...",was ronald reagan a democrat,19700,description,[],"[""In his younger years, Ronald Reagan was a me...","[0,1,0,0,0,0,0]","[""http://www.history.com/topics/us-presidents/..."
2,[20-25 minutes],"{'is_selected': [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]...",how long do you need for sydney and surroundin...,19701,numeric,[],"[""Sydney, New South Wales, Australia is locate...","[0,0,0,0,1,0,0,0,0,0]","[""https://en.wikipedia.org/wiki/Geography_of_S..."
3,[$11 to $22 per square foot],"{'is_selected': [0, 0, 0, 0, 0, 0, 0, 0, 1], '...",price to install tile in shower,19702,numeric,[],"[""In regards to tile installation costs, consu...","[0,0,0,0,0,0,0,0,1]","[""http://www.improvenet.com/r/costs-and-prices..."
4,[Due to symptoms in the body],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0], 'pas...",why conversion observed in body,19703,description,[],"[""Conclusions: In adult body CT, dose to an or...","[0,0,1,0,0,0,0,0]","[""http://www.ncbi.nlm.nih.gov/pmc/articles/PMC..."
...,...,...,...,...,...,...,...,...,...
82321,[The act or action of propagating as a increas...,"{'is_selected': [1, 0, 0], 'passage_text': ['d...",meaning of propagation,102124,description,[],"[""definition of propagation the act or action ...","[1,0,0]","[""http://www.merriam-webster.com/dictionary/pr..."
82322,[Yes],"{'is_selected': [0, 0, 1, 0, 0, 0, 0, 0, 0], '...",do you have to do a phd to be a clinical psych...,102125,description,[],"[""The Path to Becoming a Psychologist. First, ...","[0,0,1,0,0,0,0,0,0]","[""http://www.capella.edu/online-psychology-deg..."
82323,[Chablis],"{'is_selected': [0, 1, 0, 0, 0, 0], 'passage_t...",what wine goes with oysters,102126,entity,[],"[""If you need a wine to pair with oysters or m...","[0,1,0,0,0,0]","[""https://www.hellovino.com/wine/pairing/shell..."
82324,[1 Lithium carbonate 150 mg capsules. Lithium ...,"{'is_selected': [0, 0, 0, 1, 0, 0, 0, 0, 0], '...",what strengths does lithium come in,102127,description,[],"[""Your doctor will want to take regular blood ...","[0,0,0,1,0,0,0,0,0]","[""http://www.webmd.com/bipolar-disorder/bipola..."


In [4]:
result['passage_text'][1]

'["In his younger years, Ronald Reagan was a member of the Democratic Party and campaigned for Democratic candidates; however, his views grew more conservative over time, and in the early 1960s he officially became a Republican. In November 1984, Ronald Reagan was reelected in a landslide, defeating Walter Mondale and his running mate Geraldine Ferraro (1935-), the first female vice-presidential candidate from a major U.S. political party.","From Wikipedia, the free encyclopedia. A Reagan Democrat is a traditionally Democratic voter in the United States, especially a white working-class Northerner, who defected from their party to support Republican President Ronald Reagan in either or both the 1980 and 1984 elections. During the 1980 election a dramatic number of voters in the U.S., disillusioned with the economic \'malaise\' of the 1970s and the presidency of Jimmy Carter (even more than, four years earlier, Liberal Republican Gerald Ford), supported former California governor (and f

In [5]:
import ast

# Convert string representation of list to actual list
result['passage_text'] = result['passage_text'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)


In [6]:
# Create an empty list to hold the exploded rows
exploded_data = []

# Iterate over each row in the DataFrame
for index, row in result.iterrows():
    query = row['query']
    passage_texts = row['passage_text']

    # Ensure passage_texts is a list and iterate over it
    if isinstance(passage_texts, list):
        for passage in passage_texts:
            exploded_data.append({'query': query, 'passage_text': passage})

# Convert the exploded data into a new DataFrame
result_exploded = pd.DataFrame(exploded_data)

# View the result
result_exploded


,query,passage_text
0,what is rba,"Since 2007, the RBA's outstanding reputation h..."
1,what is rba,The Reserve Bank of Australia (RBA) came into ...
2,what is rba,RBA Recognized with the 2014 Microsoft US Regi...
3,what is rba,The inner workings of a rebuildable atomizer a...
4,what is rba,Results-Based Accountability® (also known as R...
...,...,...
676188,what is polarity index definition,Water (H 2 O) is an example of a polar molecul...
676189,what is polarity index definition,Supplement. Molecules can either be polar or n...
676190,what is polarity index definition,Full Definition of POLARITY. 1. : the quality ...
676191,what is polarity index definition,Part of the Smart grid glossary: Also see bipo...


In [11]:
result_exploded.to_csv('extra_dict.csv')

In [12]:
result_exploded = pd.read_csv('extra_dict.csv')

In [13]:
queries = result_exploded['query'].values
passages = result_exploded['passage_text'].values

In [14]:
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar


# Step 1: Create numpy arrays for queries and passage_texts for fast access
queries = result_exploded['query'].values
passages = result_exploded['passage_text'].values

# Step 2: Create a DataFrame of all indices (for efficient random sampling)
indices = np.arange(len(queries))

# Step 3: Create an empty array for negative samples
negative_samples = np.empty_like(passages, dtype=object)

# Step 4: Efficiently generate negative samples with a progress bar
for i in tqdm(range(len(queries)), desc="Generating negative samples"):
    query = queries[i]
    # Get the index of all passages except the current query's passage
    other_indices = indices[queries != query]
    
    # Randomly select one passage from the non-associated ones
    negative_index = np.random.choice(other_indices)
    negative_samples[i] = passages[negative_index]

# Step 5: Create a DataFrame with negative samples and concatenate it
negative_samples_df = pd.DataFrame(negative_samples, columns=['negative_sample'])
result_with_negatives = pd.concat([result_exploded, negative_samples_df], axis=1)

# Now, `result_with_negatives` contains the original data with an added column for negative samples
print(result_with_negatives)



Generating negative samples:   1%|          | 3944/676193 [00:24<1:08:59, 162.42it/s]


KeyboardInterrupt: 

In [7]:
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Step 1: Create numpy arrays for queries and passage_texts for fast access
queries = result_exploded['query'].values
passages = result_exploded['passage_text'].values

# Step 2: Create a DataFrame of all indices (for efficient random sampling)
indices = np.arange(len(queries))

# Step 3: Create an empty array for negative samples
negative_samples = np.empty_like(passages, dtype=object)

# Step 4: Efficiently generate negative samples with a progress bar, using indices 20 above and below
for i in tqdm(range(len(queries)), desc="Generating negative samples"):
    # Define a range of 20 indices before and after the current index, and clip it to ensure it doesn't go out of bounds
    min_index = max(i - 20, 0)
    max_index = min(i + 20, len(queries) - 1)
    
    # Exclude the current index itself from this range
    other_indices = np.concatenate([indices[min_index:i], indices[i+1:max_index+1]])
    
    # Randomly select one passage from the range of indices 20 above and below
    negative_index = np.random.choice(other_indices)
    negative_samples[i] = passages[negative_index]

# Step 5: Create a DataFrame with negative samples and concatenate it
negative_samples_df = pd.DataFrame(negative_samples, columns=['negative_sample'])
result_with_negatives = pd.concat([result_exploded, negative_samples_df], axis=1)

Generating negative samples: 100%|██████████| 676193/676193 [00:10<00:00, 67338.54it/s]


In [23]:
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for the progress bar

# Assuming `result_exploded` DataFrame is already defined
# Step 1: Create numpy arrays for queries and passage_texts for fast access
queries = result_exploded['query'].values
passages = result_exploded['passage_text'].values

# Step 2: Create a DataFrame of all indices (for efficient random sampling)
indices = np.arange(len(queries))

# Step 3: Create an empty array for negative samples
negative_samples = np.empty_like(passages, dtype=object)

# Step 4: Create a set to keep track of used negative indices
used_negative_indices = set()

# Step 5: Generate negative samples in one pass
# Create a dictionary to map queries to their indices
query_to_indices = {}
for idx, query in enumerate(queries):
    if query not in query_to_indices:
        query_to_indices[query] = []
    query_to_indices[query].append(idx)

# Use a mask to keep track of which indices can be used for negative sampling
mask = np.ones(len(queries), dtype=bool)

for current_query, query_indices in tqdm(query_to_indices.items(), desc="Generating negative samples"):
    # Update the mask to exclude current query indices
    mask[query_indices] = False
    
    # Get available indices for negative sampling
    available_indices = np.where(mask)[0]
    
    # Check if there are available indices left
    if len(available_indices) > 0:
        # Randomly select negative samples for all occurrences of the current query
        for i in query_indices:
            negative_index = np.random.choice(available_indices)
            negative_samples[i] = passages[negative_index]
            used_negative_indices.add(negative_index)
    else:
        # If no available indices are left, assign None
        for i in query_indices:
            negative_samples[i] = None  # Or set to a default value

    # Reset the mask for the current query indices for the next iteration
    mask[query_indices] = True

# Step 6: Create a DataFrame with negative samples and concatenate it
negative_samples_df = pd.DataFrame(negative_samples, columns=['negative_sample'])
result_with_negatives = pd.concat([result_exploded, negative_samples_df], axis=1)


Generating negative samples: 100%|██████████| 82326/82326 [01:53<00:00, 723.80it/s]


In [24]:
result_with_negatives

,query,passage_text,negative_sample
0,what is rba,"Since 2007, the RBA's outstanding reputation h...","In 1642, the year Galileo died, Isaac Newton w..."
1,what is rba,The Reserve Bank of Australia (RBA) came into ...,The cost of becoming a social worker varies by...
2,what is rba,RBA Recognized with the 2014 Microsoft US Regi...,Basically sulfites (sulphites) refer to compou...
3,what is rba,The inner workings of a rebuildable atomizer a...,Surname Meanings & Origins. Learn about the or...
4,what is rba,Results-Based Accountability® (also known as R...,Snow leopards are slightly smaller than the ot...
...,...,...,...
676188,what is polarity index definition,Water (H 2 O) is an example of a polar molecul...,The procurement officer proceeds to negotiate ...
676189,what is polarity index definition,Supplement. Molecules can either be polar or n...,Best Answer: A baby starts to walk when she/he...
676190,what is polarity index definition,Full Definition of POLARITY. 1. : the quality ...,Ceramics encompass such a vast array of materi...
676191,what is polarity index definition,Part of the Smart grid glossary: Also see bipo...,Top Yearly Tsa Salary. Some professions tend t...


In [25]:
# Check for duplicates in the negative_sample column
duplicates = result_with_negatives[result_with_negatives['negative_sample'].duplicated(keep=False)]

# Print the duplicates
if not duplicates.empty:
    print("Duplicated negative samples:")
    print(duplicates)
else:
    print("No duplicated negative samples found.")


Duplicated negative samples:
                                    query  \
0                             what is rba   
1                             what is rba   
2                             what is rba   
3                             what is rba   
4                             what is rba   
...                                   ...   
676187  what is polarity index definition   
676188  what is polarity index definition   
676189  what is polarity index definition   
676190  what is polarity index definition   
676191  what is polarity index definition   

                                             passage_text  \
0       Since 2007, the RBA's outstanding reputation h...   
1       The Reserve Bank of Australia (RBA) came into ...   
2       RBA Recognized with the 2014 Microsoft US Regi...   
3       The inner workings of a rebuildable atomizer a...   
4       Results-Based Accountability® (also known as R...   
...                                                   ...   
676

In [26]:
result_with_negatives

,query,passage_text,negative_sample
0,what is rba,"Since 2007, the RBA's outstanding reputation h...","In 1642, the year Galileo died, Isaac Newton w..."
1,what is rba,The Reserve Bank of Australia (RBA) came into ...,The cost of becoming a social worker varies by...
2,what is rba,RBA Recognized with the 2014 Microsoft US Regi...,Basically sulfites (sulphites) refer to compou...
3,what is rba,The inner workings of a rebuildable atomizer a...,Surname Meanings & Origins. Learn about the or...
4,what is rba,Results-Based Accountability® (also known as R...,Snow leopards are slightly smaller than the ot...
...,...,...,...
676188,what is polarity index definition,Water (H 2 O) is an example of a polar molecul...,The procurement officer proceeds to negotiate ...
676189,what is polarity index definition,Supplement. Molecules can either be polar or n...,Best Answer: A baby starts to walk when she/he...
676190,what is polarity index definition,Full Definition of POLARITY. 1. : the quality ...,Ceramics encompass such a vast array of materi...
676191,what is polarity index definition,Part of the Smart grid glossary: Also see bipo...,Top Yearly Tsa Salary. Some professions tend t...


In [114]:
result_with_negatives = pd.read_csv('results_negative.csv')

In [115]:
df = result_with_negatives

In [92]:
# Function to clean text
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove everything except letters and spaces
    text = re.sub(r"[^a-zA-Z\s]", "", text)  # This removes digits, punctuation, and special characters

    # Optionally: Maintain specific abbreviations by replacing them after cleaning
    text = re.sub(r"\brba\b", "rba", text)  # Add more abbreviations as needed

    return text

# Apply the clean_text function to each of the relevant columns
df['query'] = df['query'].apply(clean_text)
df['passage_text'] = df['passage_text'].apply(clean_text)
df['negative_sample'] = df['negative_sample'].apply(clean_text)

df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,query,passage_text,negative_sample
0,0,0,0,what is rba,since the rba s outstanding reputation been af...,unaffiliated the majority religion in chile in...
1,1,1,1,what is rba,the reserve bank of rba came into being on as ...,days on average range days for the infectious ...
2,2,2,2,what is rba,rba with the us regional partner of the by pr ...,a citrus tree that does not get enough water b...
3,3,3,3,what is rba,the inner of a atomizer are surprisingly simpl...,or are herbal that promote greater density and...
4,4,4,4,what is rba,based accountability also known as rba is a wa...,find by star from the big dipper if you re fam...
...,...,...,...,...,...,...
676183,676183,676188,676188,what is polarity index definition,water h o is an example of a polar molecule si...,size relative to a tea cup the porcupine is th...
676184,676184,676189,676189,what is polarity index definition,supplement can either be polar or nonpolar dep...,average air hostess the average salary for air...
676185,676185,676190,676190,what is polarity index definition,full definition of polarity the quality or con...,one of s ranging from to hip hop in june or ea...
676186,676186,676191,676191,what is polarity index definition,part of the smart grid glossary also see bipol...,days of hot weather in the temperature at into...


In [83]:
# Optionally, drop rows where cleaned text ends up empty
result_with_negatives = result_with_negatives[result_with_negatives['query'] != '']
result_with_negatives = result_with_negatives[result_with_negatives['passage_text'] != '']
result_with_negatives = result_with_negatives[result_with_negatives['negative_sample'] != '']

In [84]:
result_with_negatives

,Unnamed: 0.1,Unnamed: 0,query,passage_text,negative_sample
0,0,0,what is rba,since the rba s outstanding reputation been af...,unaffiliated the majority religion in chile in...
1,1,1,what is rba,the reserve bank of rba came into being on as ...,days on average range days for the infectious ...
2,2,2,what is rba,rba with the us regional partner of the by pr ...,a citrus tree that does not get enough water b...
3,3,3,what is rba,the inner of a atomizer are surprisingly simpl...,or are herbal that promote greater density and...
4,4,4,what is rba,based accountability also known as rba is a wa...,find by star from the big dipper if you re fam...
...,...,...,...,...,...
676183,676188,676188,what is polarity index definition,water h o is an example of a polar molecule si...,size relative to a tea cup the porcupine is th...
676184,676189,676189,what is polarity index definition,supplement can either be polar or nonpolar dep...,average air hostess the average salary for air...
676185,676190,676190,what is polarity index definition,full definition of polarity the quality or con...,one of s ranging from to hip hop in june or ea...
676186,676191,676191,what is polarity index definition,part of the smart grid glossary also see bipol...,days of hot weather in the temperature at into...


In [93]:
# Function to identify and count symbols in a text
def identify_symbols(text) -> collections.Counter:
    # If text is a list, join it into a single string
    if isinstance(text, list):
        text = ' '.join(text)
    
    # Use regex to find non-alphabetic characters (excluding spaces)
    symbols = re.findall(r'[^a-zA-Z\s]', text)
    
    # Count the occurrences of each symbol
    symbol_counts = collections.Counter(symbols)
    
    return symbol_counts

# Apply identify_symbols to each column and store the results
symbol_counts_query = identify_symbols(' '.join(df['query']))
symbol_counts_passage = identify_symbols(' '.join(df['passage_text']))
symbol_counts_negative = identify_symbols(' '.join(df['negative_sample']))

# Display the symbol counts for each column
print("Symbols found in 'query':", symbol_counts_query)
print("Symbols found in 'passage_text':", symbol_counts_passage)
print("Symbols found in 'negative_sample':", symbol_counts_negative)

Symbols found in 'query': Counter()
Symbols found in 'passage_text': Counter()
Symbols found in 'negative_sample': Counter()


In [116]:
df = result_with_negatives

In [117]:
# Step 1: Concatenate all text into a single string
all_text = ' '.join(df['query'].tolist() + df['passage_text'].tolist() + df['negative_sample'].tolist())

# Step 2: Count word frequencies
words = re.findall(r'\b\w+\b', all_text.lower())  # Split by word and ignore case
word_counts = Counter(words)

# Step 3: Get words that occur more than 15 times
words_to_remove = {word for word, count in word_counts.items() if count < 15}

# Step 4: Function to remove frequent words and single-character words
def remove_frequent_and_single_char_words(text):
    if isinstance(text, str):  # Check if the input is a string
        # Split words and filter
        return ' '.join(word for word in text.split() if word not in words_to_remove and len(word) > 1)
    return text


df['passage_text'] = df['passage_text'].apply(remove_frequent_and_single_char_words)
df['negative_sample'] = df['negative_sample'].apply(remove_frequent_and_single_char_words)

# Display the updated DataFrame
df

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,query,passage_text,negative_sample
0,0,0,0,0,0,what is rba,since the rba outstanding reputation been affe...,unaffiliated the majority religion in chile in...
1,1,1,1,1,1,what is rba,the reserve bank of rba came into being on as ...,days on average range days for the infectious ...
2,2,2,2,2,2,what is rba,rba with the us regional partner of the by pr ...,citrus tree that does not get enough water bec...
3,3,3,3,3,3,what is rba,the inner of atomizer are surprisingly simple ...,or are herbal that promote greater density and...
4,4,4,4,4,4,what is rba,based accountability also known as rba is way ...,find by star from the big dipper if you re fam...
...,...,...,...,...,...,...,...,...
676183,676183,676183,676183,676188,676188,what is polarity index definition,water is an example of polar molecule since it...,size relative to tea cup the porcupine is the ...
676184,676184,676184,676184,676189,676189,what is polarity index definition,supplement can either be polar or nonpolar dep...,average air hostess the average salary for air...
676185,676185,676185,676185,676190,676190,what is polarity index definition,full definition of polarity the quality or con...,one of ranging from to hip hop in june or each...
676186,676186,676186,676186,676191,676191,what is polarity index definition,part of the smart grid glossary also see bipol...,days of hot weather in the temperature at into...


In [118]:
df.to_csv('results_negative.csv')